In [ ]:
# 건강보험심사평가원 제공 Open API 사용 병원 현황 데이터 조회
# 진료과목코드 선택해서 해당 진료과목을 지원하는 병원 조회
# 테이블 만들기 - 테이블에 조회된 데이터 저장
# 자치구 또는 시도별 병원 현황 조회해서 파일로 저장

In [2]:
import urllib
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xml.etree import ElementTree as ET
import json

import pymysql

In [3]:
api_key = "4GAO8lWy1e5z9d7XJpsl6SmHTBzGefgEJZIW9UqrHaIoYq4TW5fHpqCqzHhln4PLQnFns/UQmvTHbLd1R/kOmg=="

url = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"

params ={
    'ServiceKey' : api_key,
    'dgsbjtCd' : '08',
    'pageNo' : '1',
    'numOfRows' : '5100'
    }

resp = requests.get(url, params = params)
xml_str = resp.content.decode('utf-8')

In [4]:
parsesd_xml = ET.fromstring(xml_str)

In [5]:
body = parsesd_xml.find('body')
items = body.find('items')
print(items)
item_list = items.findall('item')
len(item_list) , item_list[0]

<Element 'items' at 0x0000017E06E94DB0>


(5093, <Element 'item' at 0x0000017E06E94E00>)

In [6]:
data = []

for item in item_list:
    row = {}
    for child in item:# item 하위의 요소들을 순회
        #print(child.tag, child.text) 
        row[child.tag] = child.text.strip()
        row.update({child.tag:child.text.strip()})
    data.append(row)

In [7]:
df_hosp = pd.DataFrame(data)
df_hosp.head(3)

,addr,clCd,clCdNm,cmdcGdrCnt,cmdcIntnCnt,cmdcResdntCnt,cmdcSdrCnt,detyGdrCnt,detyIntnCnt,detyResdntCnt,...,postNo,sgguCd,sgguCdNm,sidoCd,sidoCdNm,telno,XPos,YPos,yadmNm,ykiho
0,"경기도 수원시 팔달구 중부대로 93, (지동)",01,상급종합,0,0,0,0,0,0,0,...,16247,310603,수원팔달구,310000,경기,031-1577-8588,127.0274271,37.2779855,가톨릭대학교 성빈센트병원,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...
1,"인천광역시 부평구 동수로 56, (부평동)",01,상급종합,0,0,0,0,1,0,0,...,21431,220003,인천부평구,220000,인천,032-1544-9004,126.7248987,37.4848309,가톨릭대학교인천성모병원,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...
2,"강원특별자치도 강릉시 사천면 방동길 38, (사천면)",01,상급종합,0,0,0,0,0,0,0,...,25440,320100,강릉시,320000,강원,033-610-3114,128.8578411,37.8184325,강릉아산병원,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...


In [8]:
print(df_hosp.columns)
df_hosp.info()
df_hosp.isnull().sum() 

Index(['addr', 'clCd', 'clCdNm', 'cmdcGdrCnt', 'cmdcIntnCnt', 'cmdcResdntCnt',
       'cmdcSdrCnt', 'detyGdrCnt', 'detyIntnCnt', 'detyResdntCnt',
       'detySdrCnt', 'drTotCnt', 'emdongNm', 'estbDd', 'hospUrl',
       'mdeptGdrCnt', 'mdeptIntnCnt', 'mdeptResdntCnt', 'mdeptSdrCnt',
       'pnursCnt', 'postNo', 'sgguCd', 'sgguCdNm', 'sidoCd', 'sidoCdNm',
       'telno', 'XPos', 'YPos', 'yadmNm', 'ykiho'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5093 entries, 0 to 5092
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   addr            5093 non-null   object
 1   clCd            5093 non-null   object
 2   clCdNm          5093 non-null   object
 3   cmdcGdrCnt      5093 non-null   object
 4   cmdcIntnCnt     5093 non-null   object
 5   cmdcResdntCnt   5093 non-null   object
 6   cmdcSdrCnt      5093 non-null   object
 7   detyGdrCnt      5093 non-null   object
 8   detyIntnCnt     50

addr                 0
clCd                 0
clCdNm               0
cmdcGdrCnt           0
cmdcIntnCnt          0
cmdcResdntCnt        0
cmdcSdrCnt           0
detyGdrCnt           0
detyIntnCnt          0
detyResdntCnt        0
detySdrCnt           0
drTotCnt             0
emdongNm            42
estbDd               0
hospUrl           3957
mdeptGdrCnt          0
mdeptIntnCnt         0
mdeptResdntCnt       0
mdeptSdrCnt          0
pnursCnt             0
postNo               0
sgguCd               0
sgguCdNm             0
sidoCd               0
sidoCdNm             0
telno               76
XPos               147
YPos               147
yadmNm               0
ykiho                0
dtype: int64

In [9]:
selected_columns = ['sidoCd', 'sidoCdNm', 'sgguCd', 'sgguCdNm', 'clCd', 'clCdNm', 'yadmNm', 'drTotCnt', 'addr', 'estbDd']

df_hospital = df_hosp[selected_columns]
df_hospital.head(3)
df_hospital.info()
df_hospital.isnull().sum() 


# sidoCd	시도코드    int
# sidoCdNm  시도명      char
# sgguCd	시군구코드  int
# sgguCdNm  시군구명    char
# clCd      종별코드    char
# clCdNm	종별코드명  char
# yadmNm	병원명      char
# drTotCnt  의사총수    char
# addr      주소        char
# telno     전화번호    char
# estbDd	개설일자    char

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5093 entries, 0 to 5092
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sidoCd    5093 non-null   object
 1   sidoCdNm  5093 non-null   object
 2   sgguCd    5093 non-null   object
 3   sgguCdNm  5093 non-null   object
 4   clCd      5093 non-null   object
 5   clCdNm    5093 non-null   object
 6   yadmNm    5093 non-null   object
 7   drTotCnt  5093 non-null   object
 8   addr      5093 non-null   object
 9   estbDd    5093 non-null   object
dtypes: object(10)
memory usage: 398.0+ KB


sidoCd      0
sidoCdNm    0
sgguCd      0
sgguCdNm    0
clCd        0
clCdNm      0
yadmNm      0
drTotCnt    0
addr        0
estbDd      0
dtype: int64

In [48]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    cursor = conn.cursor()

    sql = "drop table if exists hospital"
    cursor.execute(sql)
    sql = """ create table if not exists hospital
         ( 
              idx int primary key auto_increment,
              sidoCd int not null,
              sidoCdNm varchar(50) not null,
              sgguCd int not null,
              sgguCdNm varchar(50) not null,
              clCd varchar(50) not null,
              clCdNm varchar(50) not null,
              yadmNm varchar(50) not null,
              drTotCnt varchar(50) not null,
              addr varchar(200) not null,
              estbDd varchar(50) not null
         )"""
    cursor.execute(sql)
     
except Exception as e:
    print("오류발생 : ", e)
    if conn:
        conn.rollback() 
finally:

    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [49]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    cursor = conn.cursor()

    # 명령 실행
    sql = "delete from hospital"
    cursor.execute(sql)

    sql = """insert into hospital 
             (sidoCd, sidoCdNm, sgguCd, sgguCdNm, clCd, clCdNm, yadmNm, drTotCnt, addr, estbDd)
             values 
             (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    data = df_hospital.values.tolist()
    cursor.executemany(sql, data)

    conn.commit()

except Exception as e:
    print("오류발생 : ", e)
    if conn:
        conn.rollback()

finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [ ]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    cursor = conn.cursor()

    sql = """select sgguCdNm, count(*) cnt from hospital group by sgguCdNm order by cnt DESC"""
    cursor.execute(sql)

    hospital_data = cursor.fetchall()

    df_hospital = pd.DataFrame(hospital_data,  columns=['자치구', '병원수'] )
    df_hospital.to_csv('data-files/hospital-form-count-by-gu.csv', index=False)

except Exception as e:
    print("오류발생 : ", e)
finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [10]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    cursor = conn.cursor()

    sql = """select sidoCdNm, count(*) cnt from hospital group by sidoCdNm order by cnt DESC"""
    cursor.execute(sql)

    hospitala_data = cursor.fetchall()

    df_hospitala = pd.DataFrame(hospitala_data,  columns=['자치구', '병원수'] )
    df_hospitala.to_csv('data-files/hospital-form2-count-by-gu.csv', index=False)

except Exception as e:
    print("오류발생 : ", e)
finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()